In [1]:
from google.colab import files
uploaded = files.upload()


Saving hybrid_manufacturing_categorical.csv to hybrid_manufacturing_categorical.csv


In [2]:
import pandas as pd
import numpy as np


In [3]:
df = pd.read_csv('hybrid_manufacturing_categorical.csv', parse_dates=['Scheduled_Start'])
df.head()


,Job_ID,Machine_ID,Operation_Type,Material_Used,Processing_Time,Energy_Consumption,Machine_Availability,Scheduled_Start,Scheduled_End,Actual_Start,Actual_End,Job_Status,Optimization_Category
0,J001,M01,Grinding,3.17,76,11.42,96,2023-03-18 08:00:00,2023-03-18 09:16:00,2023-03-18 08:05:00,2023-03-18 09:21:00,Completed,Moderate Efficiency
1,J002,M01,Grinding,3.35,79,6.61,84,2023-03-18 08:10:00,2023-03-18 09:29:00,2023-03-18 08:20:00,2023-03-18 09:39:00,Delayed,Low Efficiency
2,J003,M04,Additive,2.29,56,11.11,92,2023-03-18 08:20:00,2023-03-18 09:16:00,NaN,NaN,Failed,Low Efficiency
3,J004,M04,Grinding,1.76,106,12.50,95,2023-03-18 08:30:00,2023-03-18 10:16:00,2023-03-18 08:35:00,2023-03-18 10:21:00,Completed,Moderate Efficiency
4,J005,M01,Lathe,1.90,46,8.13,88,2023-03-18 08:40:00,2023-03-18 09:26:00,2023-03-18 08:42:00,2023-03-18 09:28:00,Completed,High Efficiency


In [4]:
# Map Machine_ID to Product IDs & Names
product_map = {
    'M01': 'P001',
    'M02': 'P002',
    'M03': 'P003',
    'M04': 'P004',
    'M05': 'P005'
}
product_name_map = {
    'P001': 'Gear Shaft',
    'P002': 'Bearing Set',
    'P003': 'Drive Belt',
    'P004': 'Hydraulic Pump',
    'P005': 'Valve Assembly'
}

df['product_id'] = df['Machine_ID'].map(product_map)
df['product_name'] = df['product_id'].map(product_name_map)

# Keep only rows we mapped
df = df[df['product_id'].notnull()]

# Get week start date
df['week_start'] = df['Scheduled_Start'].dt.to_period('W').apply(lambda r: r.start_time.date())

# Material_Used as demand proxy
df['units_sold'] = df['Material_Used'].round().astype(int)

# Aggregate weekly by product
weekly_demand = df.groupby(['week_start', 'product_id', 'product_name'])['units_sold'].sum().reset_index()

# Simulate inventory
inventory_levels = {}
inventory_on_hand = []
lead_time_choices = [7, 14, 21]

for idx, row in weekly_demand.iterrows():
    pid = row['product_id']
    demand = row['units_sold']
    if pid not in inventory_levels:
        inventory_levels[pid] = 1000  # Initial inventory
    inventory_levels[pid] -= demand
    if inventory_levels[pid] < 200:
        inventory_levels[pid] += 500  # Replenishment
    inventory_on_hand.append(inventory_levels[pid])

weekly_demand['inventory_on_hand'] = inventory_on_hand
weekly_demand['lead_time_days'] = np.random.choice(lead_time_choices, size=len(weekly_demand))

# Season function
def get_season(date):
    m = pd.to_datetime(date).month
    if m in [12, 1, 2]:
        return "Winter"
    elif m in [3, 4, 5]:
        return "Spring"
    elif m in [6, 7, 8]:
        return "Summer"
    else:
        return "Autumn"

weekly_demand['season'] = weekly_demand['week_start'].apply(get_season)

# Promotions flag and unit prices
weekly_demand['promotions'] = np.where(np.random.rand(len(weekly_demand)) < 0.1, 1, 0)
price_map = {'P001': 15.5, 'P002': 20.0, 'P003': 10.0, 'P004': 25.0, 'P005': 18.0}
weekly_demand['unit_price'] = weekly_demand['product_id'].map(price_map)
weekly_demand['unit_price'] = weekly_demand['unit_price'] * (1 + (np.random.rand(len(weekly_demand)) - 0.5) * 0.1)
weekly_demand['unit_price'] = weekly_demand['unit_price'].round(2)

# Save final tailored CSV
weekly_demand.to_csv('tailored_demand_inventory.csv', index=False)
weekly_demand.head(10)


,week_start,product_id,product_name,units_sold,inventory_on_hand,lead_time_days,season,promotions,unit_price
0,2023-03-13,P001,Gear Shaft,142,858,7,Spring,0,15.03
1,2023-03-13,P002,Bearing Set,163,837,14,Spring,0,19.81
2,2023-03-13,P003,Drive Belt,142,858,14,Spring,0,9.88
3,2023-03-13,P004,Hydraulic Pump,147,853,7,Spring,0,25.58
4,2023-03-13,P005,Valve Assembly,135,865,21,Spring,0,18.60
5,2023-03-20,P001,Gear Shaft,462,396,7,Spring,0,15.34
6,2023-03-20,P002,Bearing Set,478,359,14,Spring,0,20.62
7,2023-03-20,P003,Drive Belt,420,438,21,Spring,0,9.71
8,2023-03-20,P004,Hydraulic Pump,455,398,14,Spring,0,24.75
9,2023-03-20,P005,Valve Assembly,472,393,14,Spring,0,17.14


In [5]:
files.download('tailored_demand_inventory.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>